# Preliminary Model Build for Mockup

In [ ]:
# Import Warnings and Dependencies
import warnings
warnings.filterwarnings('ignore')
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib
from pathlib import Path
from collections import Counter



In [ ]:
# import sklearn functions as needed 2022.11.1 Load then remove
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

### Data to Load

In [ ]:
# Load in Data
file_path = Path('data.csv') # Replace this with summary data from SQL Database and drive the connection to 
housing_data_df = pd.read_csv(file_path)

# ? Drop any Null Values or Rows or Replace ? Cleaning should be done with SQL or do we want to manipulate here ?


### Split the Data Into Training and Testing

In [ ]:
# Create our features
X = housing_data_df.drop(columns="<Select all but Target>") # TBD based on factors to pull in
y = housing_data_df["<Insert Target Column>"]

In [ ]:
# Import sklearn model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)
# Create the scaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
#Fitting the scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Determine the Model Type
Will need to evaluate the type based on accuracy and balance reports

Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOversampler
# Implement Random Oversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)
# Fit the model
classifier.fit(X_resampled,y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = classifier.predict(X_test_scaled)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test,y_pred))